## Library imports & helper functions

In [ ]:
%pip install --upgrade py2neo

In [ ]:
from py2neo import Graph
from py2neo import Node, Relationship

In [ ]:
import json

def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [ ]:
import configparser

def read_ini_file(file_path):
  config = configparser.ConfigParser()
  config.read(file_path)
  ini_dict = { section: dict(config.items(section))
               for section in config.sections() }
  return ini_dict

## Parameters -- change this accordingly!

In [ ]:
delete_db = True
write_db = True
file_path = '../sample_output/jhotdraw_detailed.json'

secrets = read_ini_file('secrets.ini')

## Connect to neo4j

In [ ]:
graph = Graph(secrets['neo4j']['url'], auth=(secrets['neo4j']['username'], secrets['neo4j']['password']))

## Delete database content

In [ ]:
# Delete everything in the DB
# **Warning:** only for starting over in case of inconsistent DB content.
if delete_db:
  graph.delete_all()

## Read file, create nodes & edges

In [ ]:
if write_db:
  data = read_json_file(file_path)

In [ ]:
if write_db:
  node_dict = { node['data']['id']: Node(
    *node['data']['labels'], 
    name=node['data']['properties']['simpleName'], 
    **node['data']['properties'], 
    qualifiedName=node['data']['id']) for node in data['elements']['nodes'] }
node_dict

In [ ]:
if write_db:
  edge_dict = { edge['data']['id']: Relationship(
    node_dict[edge['data']['source']], 
    edge['data']['label'], 
    node_dict[edge['data']['target']], 
    **edge['data']['properties']) for edge in data['elements']['edges'] }
edge_dict

## Write nodes & edges to database

In [ ]:
if write_db:
  tx = graph.begin()

  for node in node_dict.values():
    tx.create(node)

  for edge in edge_dict.values():
    tx.create(edge)

  graph.commit(tx)
